In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

#os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

2024-09-13 16:33:45.240352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 16:33:45.286730: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 16:33:45.287364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 16:33:46.060046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
import pandas as pd
from keras.utils import np_utils

In [3]:
# Read in the training CSV file
print( "Reading Training csv file.")
df1 = pd.read_csv("UNSW_NB15_training-set.csv")
df1.drop('label', axis=1, inplace=True)

#One hot encoding the string variables
obj_df1=df1
obj_df1["proto"] = obj_df1["proto"].astype('category')
obj_df1["service"] = obj_df1["service"].astype('category')
obj_df1["state"] = obj_df1["state"].astype('category')
obj_df1["proto_cat"] = obj_df1["proto"].cat.codes
obj_df1["service_cat"] = obj_df1["service"].cat.codes
obj_df1["state_cat"] = obj_df1["state"].cat.codes

obj_df1["proto"] = obj_df1["proto_cat"]
obj_df1["service"] = obj_df1["service_cat"]
obj_df1["state"] = obj_df1["state_cat"]

obj_df1.drop('proto_cat', axis=1, inplace=True)
obj_df1.drop('service_cat', axis=1, inplace=True)
obj_df1.drop('state_cat', axis=1, inplace=True)

Y_train_all_attacks = obj_df1["attack_cat"]
X_train = obj_df1.values[:,:-1]

#Normalizing train set
for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_train)):
        X_train[i,j] = round(X_train[i,j]/maximum,3)

#Make a catagorical cloumn for each type of label in trainset
obj_df1=pd.get_dummies(obj_df1, columns=["attack_cat"])
Y_train_each_attach = obj_df1.values[:,-10:]
        
# Read in the testing CSV file 
print("Reading Testing csv file.")
df2 = pd.read_csv("UNSW_NB15_testing-set.csv")
df2.drop('label', axis=1, inplace=True)

#One hot encoding the string variables
obj_df2=df2
obj_df2["proto"] = obj_df2["proto"].astype('category')
obj_df2["service"] = obj_df2["service"].astype('category')
obj_df2["state"] = obj_df2["state"].astype('category')
obj_df2["proto_cat"] = obj_df2["proto"].cat.codes
obj_df2["service_cat"] = obj_df2["service"].cat.codes
obj_df2["state_cat"] = obj_df2["state"].cat.codes

obj_df2["proto"] = obj_df2["proto_cat"]
obj_df2["service"] = obj_df2["service_cat"]
obj_df2["state"] = obj_df2["state_cat"]

obj_df2.drop('proto_cat', axis=1, inplace=True)
obj_df2.drop('service_cat', axis=1, inplace=True)
obj_df2.drop('state_cat', axis=1, inplace=True)

Y_test_all_attacks = obj_df2["attack_cat"]
X_test = obj_df2.values[:,:-1]

#Normalizing test set
for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_test)):
        X_test[i,j] = round(X_test[i,j]/maximum,3)
        
#Make a catagorical cloumn for each type of label in testset
obj_df2=pd.get_dummies(obj_df2, columns=["attack_cat"])
Y_test_each_attach = obj_df2.values[:,-10:]

cleanup_nums = {"Worms":0, "Shellcode":1, "Reconnaissance":2, "Normal":3, "Generic":4, "Fuzzers":5, "Exploits":6, "DoS":7, "Backdoor":8, "Analysis":9}
Y_train_all_attacks.replace(cleanup_nums,inplace=True)
Y_test_all_attacks.replace(cleanup_nums,inplace=True)
print('Trainset size: ',X_train.shape,Y_train_all_attacks.shape,'Testset size: ',X_test.shape,Y_test_all_attacks.shape)
Y_train = Y_train_all_attacks 
Y_test = Y_test_all_attacks

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", Y_train.shape)
Y_train = np_utils.to_categorical(Y_train, n_classes)
Y_test = np_utils.to_categorical(Y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Reading Training csv file.
Reading Testing csv file.
Trainset size:  (175341, 43) (175341,) Testset size:  (82332, 43) (82332,)
Shape before one-hot encoding:  (175341,)
Shape after one-hot encoding:  (175341, 10)


In [4]:
import keras

model = Sequential()
model.add(Dense(64, input_shape=(43,), activation='relu'))
model.add(Dense(10, activation='softmax'))
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=opt)
model_losses = []
model_accs = []
class LossHistory(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.losses = []
            self.batches = []
            self.weight_save = []
            self.model_weights = model.get_weights()
            self.weight_masks = []
            for i in range(len(self.model_weights)):
                if i%2 == 0:
                    self.random_mask = np.logical_and(np.random.randint(0,2,self.model_weights[i].shape), np.random.randint(0,2,self.model_weights[i].shape))
                    for j in range(0):
                        self.random_mask = np.logical_and(self.random_mask, np.random.randint(0,2,self.model_weights[i].shape))
                    self.weight_masks.append(self.random_mask)
            for i in range(len(self.model_weights)):
                if i == 0:
                    self.model_weights[i] = np.multiply(self.model_weights[i], self.weight_masks[int(i/2)])
            model.set_weights(self.model_weights)
            self.weight_save.append(model.get_weights)       

        def on_batch_end(self, batch, logs={}):
            self.losses.append(logs.get('loss'))
            self.batches.append(batch)
            self.model_weights = model.get_weights()
            for i in range(len(self.model_weights)):
                if i %2 == 0:
                    self.model_weights[i] = np.multiply(self.model_weights[i], self.weight_masks[int(i/2)])
            model.set_weights(self.model_weights)
            self.weight_save.append(model.get_weights) 
            
l_history=LossHistory()
history = model.fit(X_train, Y_train,
          batch_size=100, epochs=10, verbose=1,
          validation_data=(X_test, Y_test), callbacks = [l_history])

loss_and_metrics = model.evaluate(X_test, Y_test, verbose=2)
model_losses.append(loss_and_metrics[0])
model_accs.append(loss_and_metrics[1])
print(model_losses, model_accs)

Epoch 1/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.7214 - accuracy: 0.7425 - val_loss: 303597920.0000 - val_accuracy: 0.3039
Epoch 2/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.6276 - accuracy: 0.7731 - val_loss: 344225888.0000 - val_accuracy: 0.3293
Epoch 3/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.6150 - accuracy: 0.7768 - val_loss: 373325824.0000 - val_accuracy: 0.3504
Epoch 4/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.6052 - accuracy: 0.7807 - val_loss: 413484384.0000 - val_accuracy: 0.1237
Epoch 5/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.5964 - accuracy: 0.7827 - val_loss: 992014208.0000 - val_accuracy: 0.1571
Epoch 6/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.5884 - accuracy: 0.7833 - val_loss: 4083260416.0000 - val_accuracy: 0.0821
Epoch 7/10
1754/1754 [==============================] - 6s 3ms/step - loss: 0.5820 - ac

In [5]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
y_keras = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))))

2573/2573 [==============================] - 2s 785us/step
Accuracy: 0.27953893990186074


In [6]:
Y_test_original = np.argmax(Y_test, axis=1)

In [7]:
# Create the C file
with open('UNSW_testdata/test_dataset.c', 'w') as f:
    # Write the data samples array
    f.write("float test_data[%d][%d] = {\n" % (X_test.shape[0], X_test.shape[1]))
    for row in X_test:
        f.write("    {%s},\n" % ", ".join(f"{x:.6f}f" for x in row))
    f.write("};\n\n")

    # Write the labels array
    f.write("int test_labels[%d] = {\n" % len(Y_test_original))
    f.write("    %s\n" % ", ".join(map(str, Y_test_original)))
    f.write("};\n")

print("Test dataset has been saved to 'test_dataset.c'")


Test dataset has been saved to 'test_dataset.c'


In [10]:
# Get the weights and biases
layer1_weights = model.layers[0].get_weights()[0].T
layer1_bias = model.layers[0].get_weights()[1]
layer2_weights = model.layers[1].get_weights()[0].T
layer2_bias = model.layers[1].get_weights()[1]

# Write weights and biases to weights.cpp
with open('UNSW_testdata/weights.cpp', 'w') as f:
    f.write('#include <vector>\n\n')
    
    f.write('std::vector<std::vector<float>> layer1_weights = {\n')
    for row in layer1_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer1_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer1_bias))
    f.write('};\n\n')

    f.write('std::vector<std::vector<float>> layer2_weights = {\n')
    for row in layer2_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer2_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer2_bias))
    f.write('};\n')

print("Weights and biases have been saved to 'weights.cpp'")


Weights and biases have been saved to 'weights.cpp'


In [11]:
!g++ -std=c++11 UNSW_testdata/test.cpp -o UNSW_testdata/test_nn

In [12]:
!./UNSW_testdata/test_nn

Accuracy: 27.9539%
